# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Set base url

GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

In [ ]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

In [ ]:
# Set models

gpt_model = "gpt-4.1-mini"
gemini_model = "gemini-2.0-flash"

In [ ]:
system_gpt_prompt = "You are an assistant with general knowledge obtained from the internet. \
Always respond with a cheerful tone. If you don’t know the answer to a question, simply say that you don’t know."

In [ ]:
system_gemini_prompt = "You are an expert translator with knowledge of all existing languages. \
Your only task is, given a provided sentence, to translate it into the specified target language. \
Do not provide anything else in your response only the translation itself."

In [ ]:
def count_letter_tool(sentence, letter):

    if len(letter) != 1:
        return "You need to provide a single letter to count"
    
    return sentence.lower().count(letter.lower())

In [ ]:
def translator_tool(sentence, language):
    user_message = f"Please translate this sentence: \"{sentence}\" to this language: {language}"
    messages = [{"role": "system", "content": system_gemini_prompt}, {"role": "user", "content":user_message}]
    response = gemini.chat.completions.create(model=gemini_model, messages=messages)

    return response.choices[0].message.content

In [ ]:
count_letter_function = {
    "name": "count_letter_tool",
    "description": "Count the number of a particular letter in a sentence. Call this whenever you need to know how many times a letter appears in a sentence, for example when a user asks 'How many 'a' are in this sentence?'",
    "parameters": {
        "type": "object",
        "properties": {
            "sentence": {
                "type": "string",
                "description": "The sentence provided by the user for counting."
            },
            "letter": {
                "type": "string",
                "description": "The letter to count in the sentence."
            }
        },
        "required": ["sentence", "letter"],
        "additionalProperties": False
    }
}

In [ ]:
translator_function = {
    "name": "translator_tool",
    "description": "Translate a sentence provided by the user. Call this whenever a translation is needed, for example when a user asks 'Can you translate \"hola como estás?\" to English?'",
    "parameters": {
        "type": "object",
        "properties": {
            "sentence": {
                "type": "string",
                "description": "The sentence provided by the user to translate."
            },
            "language": {
                "type": "string",
                "description": "The target language to translate the sentence into."
            }
        },
        "required": ["sentence", "language"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": count_letter_function}, {"type": "function", "function": translator_function}]

In [ ]:
def chat(message, history):
    messages = [{"role": "system", "content": system_gpt_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=gpt_model, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=gpt_model, messages=messages)
    
    return response.choices[0].message.content

In [ ]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    sentence = arguments.get('sentence')
    response =""
    match tool_call.function.name:
        case  "translator_tool":
            language = arguments.get('language')
            translation = translator_tool(sentence, language)
            response = {"role": "tool", "content": json.dumps({"translation": translation}), "tool_call_id": tool_call.id}
        case "count_letter_tool":
            letter = arguments.get('letter')
            count = count_letter_tool(sentence, letter)
            response = {"role": "tool", "content": json.dumps({"count": count}), "tool_call_id": tool_call.id}

    return response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()